In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Dataset “Babyboom” (переменные Time of birth recorded on the 24-hour clock, Sex of the child (1 = girl, 2 = boy), Birth weight in grams, Number of minutes after midnight of each birth): 

* Проверьте вес младенцев на нормальность. Сначала используйте все данные, не разделяя детей по полу, затем для группы мальчиков и девочек по отдельности. При проверке гипотез используйте точечные оценки параметров. Постройте доверительные интервалы для параметров нормального распределения. 

* Проверьте гипотезу о том, что время между рождением детей подчиняется экспоненциальному распределению. При проверке гипотезы используйте точечные оценки параметров. 

* Проверьте гипотезу, подчиняется ли количество рождений в час для каждого часа распределению Пуассона. При проверке гипотезы используйте точечные оценки параметров. 

   

In [54]:
# Определение ширины каждого поля
colspecs = [
    (0, 8),    # Time of birth
    (8, 16),   # Sex of the child
    (16, 24),  # Birth weight in grams
    (24, 32)   # Number of minutes after midnight of each birth
]

# Определение имен колонок
column_names = [
    "Time_of_birth", "Sex", "Birth_weight", "Minutes_after_midnight"
]

# Загрузка данных
data = pd.read_fwf('data/babyboom.dat.txt', colspecs=colspecs, header=None, names=column_names)

data

,Time_of_birth,Sex,Birth_weight,Minutes_after_midnight
0,5,1,3837,5
1,104,1,3334,64
2,118,2,3554,78
3,155,2,3838,115
4,257,2,3625,177
5,405,1,2208,245
6,407,1,1745,247
7,422,2,2846,262
8,431,2,3166,271
9,708,2,3520,428


In [55]:
from scipy import stats

# Проверка нормальности для всех данных
all_weights = data['Birth_weight']
ks_stat_all, p_value_all = stats.kstest(all_weights, 'norm', args=(all_weights.mean(), all_weights.std()))
print(f"KS Statistic (All): {ks_stat_all}, P-value (All): {p_value_all}")

# Проверка нормальности для мальчиков
boys_weights = data[data['Sex'] == 2]['Birth_weight']
ks_stat_boys, p_value_boys = stats.kstest(boys_weights, 'norm', args=(boys_weights.mean(), boys_weights.std()))
print(f"KS Statistic (Boys): {ks_stat_boys}, P-value (Boys): {p_value_boys}")

# Проверка нормальности для девочек
girls_weights = data[data['Sex'] == 1]['Birth_weight']
ks_stat_girls, p_value_girls = stats.kstest(girls_weights, 'norm', args=(girls_weights.mean(), girls_weights.std()))
print(f"KS Statistic (Girls): {ks_stat_girls}, P-value (Girls): {p_value_girls}")

# Доверительные интервалы для параметров нормального распределения
def confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, len(data) - 1)
    return mean, mean - h, mean + h

# Доверительные интервалы для всех данных
mean_all, ci_lower_all, ci_upper_all = confidence_interval(all_weights)
print(f"Confidence Interval (All): Mean={mean_all}, Lower={ci_lower_all}, Upper={ci_upper_all}")

# Доверительные интервалы для мальчиков
mean_boys, ci_lower_boys, ci_upper_boys = confidence_interval(boys_weights)
print(f"Confidence Interval (Boys): Mean={mean_boys}, Lower={ci_lower_boys}, Upper={ci_upper_boys}")

# Доверительные интервалы для девочек
mean_girls, ci_lower_girls, ci_upper_girls = confidence_interval(girls_weights)
print(f"Confidence Interval (Girls): Mean={mean_girls}, Lower={ci_lower_girls}, Upper={ci_upper_girls}")

KS Statistic (All): 0.18336357215784677, P-value (All): 0.09106523162555868
KS Statistic (Boys): 0.15544307065675722, P-value (Boys): 0.5070763720555325
KS Statistic (Girls): 0.21427812399671214, P-value (Girls): 0.3315835424835907
Confidence Interval (All): Mean=3275.9545454545455, Lower=3115.418005028038, Upper=3436.491085881053
Confidence Interval (Boys): Mean=3375.3076923076924, Lower=3202.4161508132743, Upper=3548.1992338021105
Confidence Interval (Girls): Mean=3132.4444444444443, Lower=2818.3658395570424, Upper=3446.5230493318463


In [56]:
# Вычисление времени между рождением детей
time_between_births = data['Minutes_after_midnight'].diff().dropna()

# Проверка гипотезы о экспоненциальном распределении
lambda_est = 1 / time_between_births.mean()
ks_stat_exp, p_value_exp = stats.kstest(time_between_births, 'expon', args=(0, 1/lambda_est))

print(f"KS Statistic (Exponential): {ks_stat_exp}, P-value (Exponential): {p_value_exp}")

KS Statistic (Exponential): 0.13751184043266534, P-value (Exponential): 0.3574267136129613


In [57]:
# Подсчет количества рождений в каждом часе
data['Hour'] = data['Minutes_after_midnight'] // 60
births_per_hour = data['Hour'].value_counts().sort_index()

# Оценка параметра λ для распределения Пуассона
lambda_poisson = births_per_hour.mean()

# Проверка гипотезы о распределении Пуассона
ks_stat_poisson, p_value_poisson = stats.kstest(births_per_hour, 'poisson', args=(lambda_poisson,))

print(f"KS Statistic (Poisson): {ks_stat_poisson}, P-value (Poisson): {p_value_poisson}")

KS Statistic (Poisson): 0.38084101931915376, P-value (Poisson): 0.003010055985884663


___
Dataset “Euroweight” (переменные weight, batch): 

* Проверить гипотезу о нормальном распределении веса монет (сначала объединить все монеты в одну выборку, потом по каждому пакету в отдельности). При проверке гипотез используйте точечные оценки параметров.  

* Постройте доверительные интервалы для параметров нормального распределения. 




In [58]:
# Определение имен колонок
column_names = [
    "ID", "Weight", "Batch"
]

# Загрузка данных из переменной lines
data = pd.read_csv('data/euroweight.dat.txt', sep='\t', header=None, names=column_names)

data

,ID,Weight,Batch
0,1,7.512,1
1,2,7.502,1
2,3,7.461,1
3,4,7.562,1
4,5,7.528,1
...,...,...,...
1995,1996,7.514,8
1996,1997,7.519,8
1997,1998,7.606,8
1998,1999,7.547,8


In [59]:
# Проверка нормальности для всех данных
all_weights = data['Weight']
ks_stat_all, p_value_all = stats.kstest(all_weights, 'norm', args=(all_weights.mean(), all_weights.std()))
print(f"KS Statistic (All): {ks_stat_all}, P-value (All): {p_value_all}")

# Проверка нормальности для каждого пакета
batches = data['Batch'].unique()
for batch in batches:
    batch_weights = data[data['Batch'] == batch]['Weight']
    ks_stat_batch, p_value_batch = stats.kstest(batch_weights, 'norm', args=(batch_weights.mean(), batch_weights.std()))
    print(f"KS Statistic (Batch {batch}): {ks_stat_batch}, P-value (Batch {batch}): {p_value_batch}")

KS Statistic (All): 0.023353803446905852, P-value (All): 0.22192512324841585
KS Statistic (Batch 1): 0.038472438278823295, P-value (Batch 1): 0.8389620125716332
KS Statistic (Batch 2): 0.03265930919143112, P-value (Batch 2): 0.9444808004928708
KS Statistic (Batch 3): 0.07792267647627489, P-value (Batch 3): 0.0910506298988909
KS Statistic (Batch 4): 0.04544943866051099, P-value (Batch 4): 0.6628215726419289
KS Statistic (Batch 5): 0.03499095797591667, P-value (Batch 5): 0.9087939122045151
KS Statistic (Batch 6): 0.055099573639602606, P-value (Batch 6): 0.4186860218860704
KS Statistic (Batch 7): 0.042332335662710996, P-value (Batch 7): 0.7449928480338537
KS Statistic (Batch 8): 0.0701373917985999, P-value (Batch 8): 0.16294891063512817


In [60]:
# Доверительные интервалы для всех данных
mean_all, ci_lower_all, ci_upper_all = confidence_interval(all_weights)
print(f"Confidence Interval (All): Mean={mean_all}, Lower={ci_lower_all}, Upper={ci_upper_all}")

# Доверительные интервалы для каждого пакета
for batch in batches:
    batch_weights = data[data['Batch'] == batch]['Weight']
    mean_batch, ci_lower_batch, ci_upper_batch = confidence_interval(batch_weights)
    print(f"Confidence Interval (Batch {batch}): Mean={mean_batch}, Lower={ci_lower_batch}, Upper={ci_upper_batch}")

Confidence Interval (All): Mean=7.5212325, Lower=7.519724882523983, Upper=7.522740117476017
Confidence Interval (Batch 1): Mean=7.519656, Lower=7.515375800837306, Upper=7.523936199162694
Confidence Interval (Batch 2): Mean=7.523168000000001, Lower=7.518747761295764, Upper=7.527588238704238
Confidence Interval (Batch 3): Mean=7.509544, Lower=7.504930027624747, Upper=7.514157972375253
Confidence Interval (Batch 4): Mean=7.531104, Lower=7.5274397498218875, Upper=7.534768250178113
Confidence Interval (Batch 5): Mean=7.531396000000001, Lower=7.527705750509117, Upper=7.535086249490885
Confidence Interval (Batch 6): Mean=7.5152399999999995, Lower=7.511076559213748, Upper=7.519403440786251
Confidence Interval (Batch 7): Mean=7.523015999999999, Lower=7.518905563172776, Upper=7.527126436827222
Confidence Interval (Batch 8): Mean=7.516736, Lower=7.512205019334981, Upper=7.521266980665018


___
Dataset “iris.txt” (прочитайте описание данных в файле «iris_description.txt», переменные sepal length, sepal width, petal length, petal width, class): 

* Проверить гипотезу о нормальном распределении длины цветков, сгруппировав их по типу ириса. При проверке гипотез используйте точечные оценки параметров.  

* Постройте доверительные интервалы для параметров нормального распределения. 

In [61]:
data = pd.read_csv('data/iris.txt', sep=',', header=None, names=["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Class"])

data

,SepalLength,SepalWidth,PetalLength,PetalWidth,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [62]:
# Проверка нормальности для длины цветков, сгруппированных по типу ириса
iris_types = data['Class'].unique()
for iris_type in iris_types:
    iris_lengths = data[data['Class'] == iris_type]['SepalLength']
    ks_stat_iris, p_value_iris = stats.kstest(iris_lengths, 'norm', args=(iris_lengths.mean(), iris_lengths.std()))
    print(f"KS Statistic (Iris {iris_type}): {ks_stat_iris}, P-value (Iris {iris_type}): {p_value_iris}")

KS Statistic (Iris Iris-setosa): 0.11485990669608126, P-value (Iris Iris-setosa): 0.4889236515009082
KS Statistic (Iris Iris-versicolor): 0.09624091191683509, P-value (Iris Iris-versicolor): 0.7071234035717402
KS Statistic (Iris Iris-virginica): 0.11503428728233644, P-value (Iris Iris-virginica): 0.48699833606145904


In [63]:
# Доверительные интервалы для длины цветков, сгруппированных по типу ириса
for iris_type in iris_types:
    iris_lengths = data[data['Class'] == iris_type]['SepalLength']
    mean_iris, ci_lower_iris, ci_upper_iris = confidence_interval(iris_lengths)
    print(f"Confidence Interval (Iris {iris_type}): Mean={mean_iris}, Lower={ci_lower_iris}, Upper={ci_upper_iris}")

Confidence Interval (Iris Iris-setosa): Mean=5.006, Lower=4.905823539299264, Upper=5.106176460700737
Confidence Interval (Iris Iris-versicolor): Mean=5.936, Lower=5.789305783106826, Upper=6.082694216893174
Confidence Interval (Iris Iris-virginica): Mean=6.587999999999998, Lower=6.407285019117501, Upper=6.768714980882495
